In [ ]:
# 
pip install tensorflow
pip install keras
pip install keras-utils
pip install callbacks
pip install keras-models

pip install keras-octave-conv
pip install python-fit

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
 
import zipfile 

In [ ]:
  
with zipfile.ZipFile("brain tumour ds.zip","r") as zip_ref:
    zip_ref.extractall("./")

In [3]:
#  imports
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os # usefull for files or to get the directories
import math  # any math related computation
import shutil # can move my file from one folder to other
import glob #so i don't have to write entire path

## count the number of images is the repective  classes 0  means Brain tumour and 1 means healthy

In [6]:
ROOT_DIR = "Brain Tumor Data Set"
number_of_images = {} # creating dictionary key would name of folder and number of images of brain tumour

for dir in os.listdir(ROOT_DIR):
    number_of_images[dir] = len(os.listdir(os.path.join(ROOT_DIR,dir)))


In [7]:
number_of_images.items()

dict_items([('Brain Tumor', 17), ('Healthy', 16)])

## now split the data such that 
 -  70% for training
 - 15% for validation
 - 15% for testing

In [8]:
def dataFolder(path, split):
    #now creating training folder


    if not os.path.exists("./"+path):
        os.mkdir("./"+ path)


        for dir in os.listdir(ROOT_DIR):
            os.makedirs("./"+path+"/"+dir)

            for img in np.random.choice( a = os.listdir(os.path.join(ROOT_DIR,dir)),
                                        size=(math.floor(split*number_of_images[dir])-5)
                                        ,replace=False):
                O = os.path.join(ROOT_DIR, dir , img)  # path 
                D = os.path.join("./"+path ,dir)  # destination path
                shutil.copy(O,D)
                os.remove(O)  # then remove the image which is present on O variable path

    else:
         print(f"{path} folder is already exists")

    

In [9]:
dataFolder("train",0.7)

train folder is already exists


In [10]:
dataFolder("val",0.15) # validation folder 15%

val folder is already exists


In [11]:
dataFolder("test",0.15) # validation folder 15%

test folder is already exists


## Model creating below

In [12]:
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten , Dense ,BatchNormalization, GlobalAvgPool2D,Activation
from keras.models import Sequential
from   keras.preprocessing.image import ImageDataGenerator
import keras

In [13]:
# CNN MODEL
import tensorflow as tf

model = tf.keras.models.Sequential()

model.add(Conv2D(filters=16, kernel_size=(3,3), activation = 'relu', input_shape=(224,224,3) ))

model.add(Conv2D(filters=36,kernel_size = (3,3), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2)))

model.add(Conv2D(filters=64,kernel_size = (3,3), activation = 'relu'))
model.add(MaxPool2D(pool_size=(2, 2)))


model.add(Conv2D(filters=128, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Dropout(rate=0.25))

model.add(Flatten())
model.add(Dense(units=64, activation='relu'))

model.add(Dropout(rate=0.25))
model.add(Dense(units=1,activation='sigmoid'))



model.compile(optimizer='adam', loss= keras.losses.binary_crossentropy, metrics=['accuracy'])






In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 16)      448       
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 36)      5220      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 110, 110, 36)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 108, 108, 64)      20800     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 52, 52, 128)       7

## Preparing our image data using data generator

In [15]:
def preprocessingImages1(path):
    """ 
    input is path
    output is pre processed images
    """
    image_data = ImageDataGenerator(zoom_range= 0.2, shear_range= 0.2, rescale = 1./255, horizontal_flip = True) #data augmentation
   
    image = image_data.flow_from_directory(directory = path, target_size= (244,244), batch_size=32, class_mode = 'binary')
    return image

In [16]:
path = "train"
train_data = preprocessingImages1(path)

Found 3209 images belonging to 2 classes.


In [17]:
train_data.class_indices

{'Brain Tumor': 0, 'Healthy': 1}

In [19]:
def preprocessingImages2(path):
    """ input is path
    output is pre processed images
    """
    image_data = ImageDataGenerator( rescale = 1./255 )
    image = image_data.flow_from_directory(directory= path, target_size= (244,244,3), batch_size=32, class_mode = 'binary')
    return image

In [20]:
path = 'test'
test_data = preprocessingImages2(path)


Found 679 images belonging to 2 classes.


In [21]:
path = 'val'
val_data = preprocessingImages2(path)


Found 679 images belonging to 2 classes.


In [23]:
# Early stopping and model check point

from keras.callbacks import ModelCheckpoint, EarlyStopping

# Early Stopping
es = EarlyStopping(monitor="val_accuracy", min_delta=0.01, patience=5 , verbose=1 , mode= 'auto')

# Model Checkpoint
mc = ModelCheckpoint(monitor="val_accuracy", filepath="./bestmodel.h5", verbose=1, save_best_only= True, mode= 'auto')

cd = [es, mc]


Model Training


In [26]:
# verbose jei execution hocche ta display koray

hs = model.fit_generator(generator=test_data,
                          steps_per_epoch = 8,
                         epochs = 30,
                         verbose = 1,     
                         validation_data= val_data,
                          validation_steps= 16, 
                          callbacks = cd
                      )


 
 

NameError: name 'generator' is not defined